<a href="https://colab.research.google.com/github/mapsguy/programming-gemini/blob/main/JSON_EnumConstraints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

In [2]:
#import the genai library
from google import genai

In [3]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [4]:
model_name = "models/gemini-2.5-flash-preview-05-20"

In [50]:
#step 3: ensuring valid JSON output

#ensure types import
from google.genai import types

#set response_mime_type to 'application/json'
#provide response_schema: defines JSON output structure

response = client.models.generate_content(
    model = model_name,
    contents = "Extract the name and quantity of all items mentioned: 3 apples, 10 bananas",
    config={
        "response_mime_type":"application/json",
        #response_schema is defined below
    }
)

Defining the response schema

In [ ]:
# VERSION 1
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import google.ai.generativelanguage as glm

In [21]:
# VERSION 1: define the schema for individual items
item_schema = glm.Schema(
    type=glm.Type.OBJECT, #overall type usually OBJECT
    #properties has its own schema -> type, description
    properties={
        'name': glm.Schema(type=glm.Type.STRING, description="Name of the item"),
        'quantity': glm.Schema(type=glm.Type.INTEGER, description="Quantity of the item")
    },
    required=['name', 'quantity'] #a string array that lists property names that must be present
)

# Define the overall response schema containing a list of items
response_schema = glm.Schema(
    type=glm.Type.OBJECT, #overall type
    properties={
        'items': glm.Schema(
            type=glm.Type.ARRAY,
            description="List of items mentioned",
            items=item_schema # used because 'items' type is array; defines schema for array elements
        )
    }
)


In [27]:
# VERSION 2: Defining the schema

from pydantic import BaseModel

#defining the schema for the individual items
class Item(BaseModel):
    name: str
    quantity: int

#defining the schema for the list of items
class Response(BaseModel):
    items: list[Item]

In [48]:
# VERSION 2: Make the API call with the schema

response = client.models.generate_content(
    model = model_name,
    contents = "Extract the name and quantity of all items mentioned: 3 apples, 10 bananas, 1 orange",
    config={
        "response_mime_type":"application/json",
        "response_schema":list[Response],
    }
)

In [49]:
# Parsing the JSON Response
import json

output_json = json.loads(response.text)
print(output_json)
# Example Output: {'items': [{'name': 'apples', 'quantity': 3}, {'name': 'bananas', 'quantity': 10}, {'name': 'orange', 'quantity': 1}]}


[{'items': [{'name': 'apples', 'quantity': 3}, {'name': 'bananas', 'quantity': 10}, {'name': 'orange', 'quantity': 1}]}]


In [47]:
#step 4: enum constrained output: forcing specific choices

prompt = """
Text: This new Gemini feature is amazing!
Sentiment:
"""
#define enum schema
enum_schema = ["POSITIVE", "NEGATIVE", "NEUTRAL"]

response = client.models.generate_content(
    model = model_name,
    contents = prompt,
    config={
       "response_mime_type":"text/x.enum",
        "response_schema" : {
            "type":"STRING",
            "enum": enum_schema,
            },
        #candidate_count=1 # Ensure only one candidate is generated
    },
)

print(response.text)


POSITIVE
